<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Classification Part 2 Classification Metrics

_Authors: Matt Brems, Dave Yerrington, Noelle Brown, Jeff Hale, Ng Shu Min_

---

### Learning Objectives
- Calculate various classification model evaluation metrics.
- Describe the inverse relationship between sensitivity and specificity.
- Understand what the ROC shows and interpret ROC AUC.
---



## Importing libraries
---

We'll need the following libraries for this notebook:


In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import train_test_split.
from sklearn.model_selection import train_test_split

# Import Logistic Regression model.
#from sklearn.linear_model import LogisticRegression

# Import metrics.
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, accuracy_score, plot_roc_curve, roc_auc_score, recall_score, precision_score, f1_score

## Load in Our Model

We will load in the model that we saved previously into the pickle file:


In [ ]:
import pickle
filename = 'admissions_log_reg.pkl'

#unpacking pickle file
with open(filename, 'rb') as f:
    lr = pickle.load(f)

In [ ]:
# Load the X_test and y_test that we saved
X_test = pd.read_csv('admissions_X_test.csv', index_col=0)

# Similarly load in y_test. What is the difference when the index_col is not specified?


In [ ]:
# X_test
X_test.head()

In [ ]:
# Score the model on the test set
# What does the result represent?


## Classification Metrics

The `score` method returns the accuracy, but there are other important classification metrics that we need.

First we should store the predictions that we obtained and have a look at them more closely.

In [ ]:
# Store the predictions in a variable y_hat
y_hat = lr.predict(X_test)
y_hat

In [ ]:
# Get the probability that admit=1
pred_probs = lr.predict_proba(X_test)[:,1]

In [ ]:
predicted_results = pd.DataFrame({'gpa':X_test['gpa'], 
                                  'actual':y_test['admit'], 
                                  'predicted':y_hat,
                                  'pred_probs':pred_probs})

In [ ]:
predicted_results.head()

In [ ]:
# Show the rows where the actual is not equal to predicted
predicted_results[predicted_results['actual']!=predicted_results['predicted']]

In [ ]:
# What about where the actual is  equal to predicted

In [ ]:
# Let's see how many actual = predicted
num_correct = predicted_results[predicted_results['actual']==predicted_results['predicted']].shape[0]
num_correct

In [ ]:
# Let's see how many actual != predicted
num_wrong = predicted_results[predicted_results['actual']!=predicted_results['predicted']].shape[0]
num_wrong

In [ ]:
# Calculate the accuracy
accuracy = num_correct / (num_correct + num_wrong)
accuracy

## Confusion Matrix

Instead of manually calculating the true positive and true negatives, we can get the confusion matrix from `sklearn`.


In [ ]:
cm = confusion_matrix(y_test, y_hat)

In [ ]:
cm

## Confusion DataFrame
---

The confusion matrix we just created isn't very explanatory, so let's drop it into a pandas `DataFrame`.

In [ ]:
cm_df = pd.DataFrame(cm, columns=['pred 0', 'pred 1'], index=['actual 0', 'actual 1'])
cm_df

In [ ]:
# Calculate Accuracy
# (TP + TN) / (Total)


In [ ]:
# can also find using sklearn.metrics function
accuracy_score(y_test, y_hat)

## How many Type I errors are there?
---

<details>
    <summary>Need a hint?</summary>
    Type I = False positive
</details>

## How many Type II errors are there?
---
<details>
    <summary>Need a hint?</summary>
    Type II = False negatives
</details>

## Calculate recall (a.k.a. sensitivity)
---

<details>
    <summary>Need a hint?</summary>
    Recall = Sensitivity, and there are no p's in sensitivity: TP/P
</details>

In [ ]:
# Manual calculation


In [ ]:
# Check using sklearn.metric's recall_score


## Calculate the specificity
---

<details>
    <summary>Need a hint?</summary>
    There is a p in specificity, therefore there are no p's in the calculation: TN/all N
</details>

In [ ]:
# Manual calculation


## Calculate the precision
---

<details>
    <summary>Need a hint?</summary>
    Precision is number true positives in positive predictions: TN / (TP + FP)
</details>

In [ ]:
# Using sklearn.metric precision_score


### F1 score

The F1 score is the harmonic mean of precision and recall.

If you care about precision and recall roughly the same amount, F1 score is a great metric to use.

Note that even though all the metrics you’ve seen can be followed by the word score F1 always is. 🤷‍♀️

$$
2*\frac{(\text{Precision}*\text{Recall})} {(\text{Precision} + \text{Recall})}
$$

**What is the F1 Score?**

In [ ]:
# by hand
2 * (precision*recall) / (precision + recall)

In [ ]:
# from sklearn


## Other Confusion Matrix Tips and Tricks

In [ ]:
# plot it!
# https://github.com/justmarkham/scikit-learn-tips/blob/master/notebooks/20_plot_confusion_matrix.ipynb
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(lr, X_test, y_test, cmap='Blues', 
                      values_format='d', display_labels=['admit = 0', 'admit = 1']);

In [ ]:
## Ravel it!
# Save TN/FP/FN/TP values.

tn, fp, fn, tp = confusion_matrix(y_test, y_hat).ravel()

# Note that .ravel() will arrange items in a one-dimensional array.
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.ravel.html

## Relationship between Sensitivity and Specificity

Let's create a plot to compare the predicted probabilities and the actual results. 

In [ ]:
plt.figure(figsize = (12, 5))

sns.histplot(predicted_results, x='pred_probs', hue='actual', bins=20, alpha=0.2)
plt.xlabel('Predicted Probability that Outcome = 1')
plt.show()

Where there are overlaps, there would be misclassifications of some of the observations.

Consider if the cutoff for the predicted probability is 0.5, so an observation with predicted probability 0.5 and above would be predicted as 1, and below 0.5 would be predicted as 0.

In [ ]:
plt.figure(figsize = (12, 5))

# plot distributions of predicted probabilities by actual values
sns.histplot(predicted_results, x='pred_probs', hue='actual', bins=20, alpha=0.2)

# Add cutoff line
plt.axvline(0.5, color = 'black', linestyle = '--')

plt.xlabel('Predicted Probability that Outcome = 1')
plt.show()

In [ ]:
plt.figure(figsize = (12, 5))

# plot distributions of predicted probabilities by actual values
sns.histplot(predicted_results, x='pred_probs', hue='actual', bins=20, alpha=0.2)

# Add cutoff line
plt.axvline(0.5, color = 'black', linestyle = '--')

# Add annotations for TN, FN, TP, FP.
plt.annotate(xy = (0.0, 100), text = 'TN', size = 15)
plt.annotate(xy = (0.4, 5), text = 'FN', size = 15)
plt.annotate(xy = (0.90, 1), text = 'TP', size = 15)
plt.annotate(xy = (0.65, 10), text = 'FP', size = 15)

plt.xlabel('Predicted Probability that Outcome = 1')
plt.show()

In [ ]:
plt.figure(figsize = (12, 5))

# plot distributions of predicted probabilities by actual values
sns.histplot(predicted_results, x='pred_probs', hue='actual', bins=20, alpha=0.2)

# Add cutoff line
plt.axvline(0.3, color = 'black', linestyle = '--')

# Add annotations for TN, FN, TP, FP.
plt.annotate(xy = (0.0, 100), text = 'TN', size = 15)
plt.annotate(xy = (0.1, 1), text = 'FN', size = 15)
plt.annotate(xy = (0.90, 1), text = 'TP', size = 15)
plt.annotate(xy = (0.3, 10), text = 'FP', size = 15)


plt.xlabel('Predicted Probability that Outcome = 1')


<details><summary>When I moved my classification threshold to the left, what happened to sensitivity and specificity?</summary>
- Our number of true negatives decreased and our number of total negatives remains the same.
    - $\text{Specificity} = \frac{TN}{N} \Rightarrow \text{Specificity decreases.}$
- Our number of true positives increased and our number of total positives remains the same.
    - $\text{Sensitivity} = \frac{TP}{P} \Rightarrow \text{Sensitivity increases.}$

</details>

Our goal should be to build a model such that there is **no overlap** between the blue histogram and the orange histogram!
- If there is overlap, we need to recognize the tradeoff between sensitivity and specificity. (As one increases, the other decreases.)
- One measure of how much overlap exists between our distributions is the **area under the ROC curve**.

## Receiver Operating Characteristic (ROC) Curve

Plot the True Positive Rate vs. False Positive Rate for the range of possible decision thresholds and you get the ROC curve!

In [ ]:
plot_roc_curve(lr, X_test, y_test)
plt.plot([0, 1], [0, 1],
         label='baseline', linestyle='--')
plt.legend()
plt.show()

### ROC AUC

The more area under this blue curve is, the better separated our distributions are.
- Check out this gif ([source](https://twitter.com/DrHughHarvey/status/1104435699095404544)):

![](https://media.giphy.com/media/H1SZ5oRLIuZ1t1c4Di/giphy.gif)

We use the **area under the ROC curve** (abbreviated **ROC AUC** or **AUC ROC**) to quantify the gap between our distributions.

In [ ]:
roc_auc_score(y_test, y_hat)

### Interpreting ROC AUC
- If you have an ROC AUC of 0.5, your positive and negative populations perfectly overlap and your model is as bad as it can get.
- If you have an ROC AUC of 1, your positive and negative populations are perfectly separated and your model is as good as it can get.
- The closer your ROC AUC is to 1, the better. (1 is the maximum score.)
- If you have an ROC AUC of below 0.5, your positive and negative distributions have flipped sides. By flipping your predicted values (i.e. flipping predicted 1s and 0s), your ROC AUC will now be above 0.5.
    - Example: You have an ROC AUC of 0.2. If you change your predicted 1s to 0s and your predicted 0s to 1s, your ROC AUC will now be 0.8!

We generate one ROC curve per model. The ROC curve is generated by varying our threshold from 0 to 1. This doesn't actually change the threshold or our original predictions, but it helps us to visualize our tradeoff between _sensitivity_ and _specificity_ and understand how well-separated our populations are.

### Changing your prediction threshold

If you want, you could change your prediction threshold to a custom value.

In [ ]:
def get_predictions(probabilities, threshold):
    return [0 if prob < threshold else 1 for prob in probabilities]

In [ ]:
print(get_predictions(lr.predict_proba(X_test)[:,1], 0.3)[:10])

In [ ]:
threshold_list = [0.25, 0.4, 0.5, 0.6, 0.75]

for threshold in threshold_list:
    preds = get_predictions(lr.predict_proba(X_test)[:,1], threshold)
    acc = accuracy_score(y_test, preds)
    print(f'Threshold: {threshold}, Accuracy: {acc}')

## Summary

Classification metrics are used to evaluate a classification model. 

We can generate a confusion matrix to view the True Positives, True Negatives, False Positives and False Negatives.

Then we use these values to calculate 
- accuracy (1 - misclassification)
- recall (aka sensitivity or true positive rate)
- specificity (1 - false positive rate)
- precision (positive predictive value)

For a binary classifier, the naive classifer would be based on classifying at random (0.5 chance of getting 1 or 0), so our model should have an accuracy that is better than 50%!